In [1]:
import requests
from bs4 import BeautifulSoup
import time
from urllib import urlencode
import selenium.webdriver
import random
from pymongo import MongoClient
import json
import boto3
import pandas as pd
from parse_clean_store import (parse_route_page, parse_user, parse_clean_user)
from bs4 import BeautifulSoup
%load_ext autoreload
%autoreload 2

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.route_html_collection
route_html_collection = db.route_html_collection
raw_data = route_html_collection.find()
df_route_html = pd.DataFrame(list(raw_data))
df_route_html.head()

,_id,html,route
0,58c87f8840b44102a84aedf9,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",30(1) Feet of Pleasure
1,58c87f9740b44102a84aedff,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",41 Feet of Pain
2,58c87fa640b44102a84aee05,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.5 Crack
3,58c87fb140b44102a84aee0b,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.6 crack
4,58c87fbc40b44102a84aee11,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",5.6 Dihedral


In [3]:
df_route_html.shape

(3626, 3)

In [8]:
client = MongoClient('mongodb://localhost:27017/')
db = client.user_html_collection
user_html_collection = db.user_html_collection
raw_data = user_html_collection.find()
df_user_html = pd.DataFrame(list(raw_data))
df_user_html.head()

,_id,html,username
0,58c87f9340b44102a84aedfd,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",adam winslow
1,58c87fa440b44102a84aee03,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",adam winslow
2,58c87fae40b44102a84aee09,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Oscar Edge
3,58c87fb640b44102a84aee0f,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Nick Drake
4,58c87fc340b44102a84aee15,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",Chris Culver


In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.ratings_collection
ratings_collection = db.ratings_collection
raw_data = ratings_collection.find()
df_ratings = pd.DataFrame(list(raw_data))
df_ratings.head()

,_id,rating,route,username
0,58c87f8c40b44102a84aedfb,[1],30(1) Feet of Pleasure,[adam winslow ]
1,58c87f9d40b44102a84aee01,[1],41 Feet of Pain,[adam winslow ]
2,58c87fac40b44102a84aee07,"[1, 1, 1, 1]",5.5 Crack,"[CHopwood , Trevor. , Rickedge , Oscar Edge ]"
3,58c87fb440b44102a84aee0d,"[3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",5.6 crack,"[Bianca P , Sandeep Kumar , Mike Brady , Tom B..."
4,58c87fbf40b44102a84aee13,"[2, 2, 2, 2, 1, 1, 1, 1, 1]",5.6 Dihedral,"[Jason Killgore , sqwirll , Dave Leydet , fort..."


In [4]:
df_ratings.shape

(3726, 4)

In [13]:
ratings_df = clean_ratings(df_ratings)
ratings_df.head()

,adam winslow,CHopwood,Trevor.,Rickedge,Oscar Edge,Bianca P,Sandeep Kumar,Mike Brady,Tom Bath,Clay Studer,...,Andrew Perkins,Mike dandy Patz,Travis England,Neilsonb,Ben Broche,Dustin Stephens,Curt Shannon,Cedar Wright,Detchy,Colin Moorhead
30(1) Feet of Pleasure,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41 Feet of Pain,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.5 Crack,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.6 crack,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.6 Dihedral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
ratings_df = ratings_df.T
ratings_df.head()

,30(1) Feet of Pleasure,41 Feet of Pain,5.5 Crack,5.6 crack,5.6 Dihedral,5.7,5.7 Face,A,A Crack,A MIGHTY WIND,...,TPMV (10% Meteorological Vinculation),Tradical,Tree Hugger,Trigger Finger,Trobability Pest,Twelve Gauge IQ,Under Arrest,Under the Influence,Unicorn Blues,Unknown
adam winslow,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CHopwood,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Trevor.,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rickedge,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oscar Edge,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
cold_start(ratings_df).head()

,30(1) Feet of Pleasure,41 Feet of Pain,5.5 Crack,5.6 crack,5.6 Dihedral,5.7,5.7 Face,A,A Crack,A MIGHTY WIND,...,Town Crier,TPMV (10% Meteorological Vinculation),Tradical,Tree Hugger,Trigger Finger,Trobability Pest,Twelve Gauge IQ,Under Arrest,Under the Influence,Unicorn Blues
adam winslow,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
CHopwood,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Trevor.,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Rickedge,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Oscar Edge,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0


In [94]:
df = cold_start(df_ratings)
df.head()

,30(1) Feet of Pleasure,41 Feet of Pain,5.5 Crack,5.6 crack,5.6 Dihedral,5.7,5.7 Face,A,A Crack,A MIGHTY WIND,...,Town Crier,TPMV (10% Meteorological Vinculation),Tradical,Tree Hugger,Trigger Finger,Trobability Pest,Twelve Gauge IQ,Under Arrest,Under the Influence,Unicorn Blues
adam winslow,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
CHopwood,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Trevor.,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Rickedge,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0
Oscar Edge,1.0,1.0,1.0,2.029412,1.444444,2.0,1.692308,NaN,1.0,3.0,...,3.545455,3.928571,4.0,3.0,4.0,3.0,4.0,2.615385,2.0,4.0


In [95]:
from sklearn.decomposition import NMF


In [96]:
df_user_html.shape

NameError: name 'df_user_html' is not defined

In [97]:
soup = BeautifulSoup(html, 'html.parser')
page_tag = soup.find('div', {'id':'rspCol800'})
# make route dict
route_dict = {}
route_dict['name'] = page_tag.find('span', {'itemprop':'itemreviewed'}).text.encode('utf-8')
route_dict['grade'] = page_tag.find('span', {'class':'rateYDS'}).text
route_stars_text = soup.find('span', {'id':'starSummaryText'}).text.split('Average: ')
route_dict['average_rating'] = route_stars_text[1][:3]

NameError: name 'html' is not defined

In [98]:
def parse_route_page(html):
    '''
    takes a soup to parse
    returns dict of route info
    '''
    soup = BeautifulSoup(html, 'html.parser')
    page_tag = soup.find('div', {'id':'rspCol800'})
    # make route dict
    route_dict = {}
    route_dict['name'] = page_tag.find('span', {'itemprop':'itemreviewed'}).text
    route_dict['grade'] = clean_grade(page_tag)
    route_dict['average_rating'] = clean_average_rating(soup)
    
    for i, td in enumerate(page_tag.find('table').findAll('td')): 
        if td.text.split(':')[0] == 'Type':
            route_type = clean_route_type(page_tag.find('table').findAll('td')[i+1])
            route_dict['route_type_list'] = route_type[0]
            route_dict['pitches'] = route_type[1]
            route_dict['height'] = route_type[2]
            route_dict['uiaa'] = route_type[3]
            route_dict['misc'] = route_type[4]
        elif td.text.split(':')[0] == 'Original':
            route_dict['original_grade'] = clean_original_grade(page_tag.find('table').findAll('td')[i+1])
        elif td.text.split(':')[0] == 'FA':
            fa = page_tag.find('table').findAll('td')[i+1].text
            route_dict['FA'] = fa
        elif td.text.split(':')[0] == 'Season':
            season = page_tag.find('table').findAll('td')[i+1].text
            route_dict['season'] = season.encode('utf-8')
        elif td.text.split(':')[0] == 'Page Views':
            page_view = page_tag.find('table').findAll('td')[i+1].text
            route_dict['page_views'] = int(page_view.encode('utf-8').replace(',',''))
        elif td.text.split(':')[0] == 'Submitted By':
            sb1, sb2 = clean_submitted_by(page_tag.find('table').findAll('td')[i+1])
            route_dict['submitted_by'] = sb1
            route_dict['submitted_on'] = sb2
            
    return route_dict


def clean_route_type(tag):
    route_type = tag.text
    route_type = route_type.encode('utf-8').split(', ')
    list_of_types = ['Trad', 'Sport', 'TR', 'Alpine', 
                    'Ice', 'Boulder', 'Aid', 'Mixed']
    # initialize variables
    route_type_list = []
    pitches = float('nan')
    height = float('nan')
    uiaa = float('nan')
    misc = float('nan')
    for item in route_type:
        if item in list_of_types:
            route_type_list.append(item)
        elif 'pitches' in item:
            pitches = item.replace(' pitches', '')
        elif 'pitch' in item:
            pitches = item.replace(' pitch','')
        elif "'" in item:
            height = float(item.replace("'",''))
        elif 'Grade' in item:
            uiaa = item
        else:
            misc = item

    return route_type_list, pitches, height, uiaa, misc


def clean_submitted_by(page_tag):
    sb = page_tag.text
    sb = sb.encode('utf-8').split(' on ')
    return sb[0], sb[1]

def clean_original_grade(page_tag):
    grade = page_tag.text
    grade = grade.encode('utf-8').split('\xa0')[1]
    grade = grade.split('French')[0]
    return grade

def clean_grade(page_tag):
    grade = page_tag.find('span', {'class':'rateYDS'}).text
    grade = grade.encode('utf-8').split('\xa0')[1]
    return grade

def clean_average_rating(soup):
    route_stars_text = soup.find('span', {'id':'starSummaryText'}).text.split('Average: ')
    average_rating = route_stars_text[1][:3]
    if 'OK' in average_rating:
        average_rating = 1.0
    elif 's' in average_rating:
        average_rating = float(average_rating[0])
    else:
        average_rating = float(average_rating)
    return average_rating

In [21]:
df_route_html['_id'].shape == df_route_html['_id'].unique().shape

True

In [3]:
dict_list = []
for _id, html in enumerate(df_route_html['html']):
    dict_list.append(parse_route_page(_id, html))

In [23]:
df_routes = pd.DataFrame(dict_list)
df_routes.head(50)

,Aid,Alpine,Boulder,FA,Ice,Mixed,Sport,TR,Trad,average_rating,...,height,id,name,original_grade,page_views,pitches,season,submitted_by,submitted_on,uiaa
0,0,0,0,"Adam Winslow, Bill Coe, Kyle Silverman 3/20/09",0,0,0,0,1,1.0,...,41.0,0,30(1) Feet of Pleasure,5.6,588,1.0,NaN,adam winslow,"Aug 18, 2010",NaN
1,0,0,0,"Adam Winslow, Ujahn Davisson, Bill Coe, Jim Op...",0,0,0,0,1,1.0,...,41.0,1,41 Feet of Pain,5.5,642,1.0,NaN,adam winslow,"Aug 18, 2010",NaN
2,0,0,0,?,0,0,0,1,1,1.0,...,35.0,2,5.5 Crack,5.5,176,NaN,Afternoon shade,CHopwood,"Sep 2, 2013",NaN
3,0,0,0,,0,0,0,1,1,2.0,...,85.0,3,5.6 crack,5.6,1263,1.0,NaN,Tom Bath,"Dec 16, 2009",NaN
4,0,0,0,Unknown,0,0,0,1,1,1.4,...,30.0,4,5.6 Dihedral,5.6,678,1.0,NaN,chrishar Hargarten,"May 19, 2008",NaN
5,0,0,0,,0,0,1,0,0,2.0,...,40.0,5,5.7,5.7,326,NaN,NaN,Keenan Waeschle,"Mar 31, 2010",NaN
6,0,0,0,,0,0,0,1,1,1.7,...,95.0,6,5.7 Face,5.7,454,1.0,NaN,Mark Goodro,"Jun 18, 2013",NaN
7,0,0,0,,0,0,1,0,0,2.0,...,95.0,7,A,5.5,18,NaN,NaN,applewood,"May 28, 2013",NaN
8,0,0,0,,0,0,0,0,1,1.0,...,NaN,8,A Crack,5.7,51,NaN,NaN,Blitzo,"Oct 18, 2006",NaN
9,0,0,0,10/13,0,0,1,0,0,3.0,...,75.0,9,A MIGHTY WIND,5.3,39,NaN,NaN,applewood,"Oct 27, 2013",NaN


In [3]:
client = MongoClient('mongodb://localhost:27017/')
db = client.routes
routes = db.routes

In [4]:
routes.insert_many(dict_list)

In [6]:
raw = routes.find()
pd.DataFrame(list(raw)).T.head(50)

,0,1,2,3,4,5,6,7,8,9,...,3717,3718,3719,3720,3721,3722,3723,3724,3725,3726
Aid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alpine,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Boulder,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FA,"Adam Winslow, Bill Coe, Kyle Silverman 3/20/09","Adam Winslow, Ujahn Davisson, Bill Coe, Jim Op...",?,,Unknown,,,,,10/13,...,"Dave Anderson, Bruce Carson, 1/73 FFA(p1): Lar...",B. Workman,Michal Rynkiewicz,Jens Klubberud- route prep Andy Fitz,"The late, great, Ryan Triplett","Greg Collum, Matt Kerns",,,"Chris Kalman, Alessandra Patz, Mike Patz",Unknown
Ice,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Mixed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sport,0,0,0,0,0,1,0,1,0,1,...,0,0,1,1,1,0,1,1,1,1
TR,0,0,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Trad,1,1,1,1,1,0,1,0,1,0,...,1,1,0,0,0,1,0,0,0,0
_id,58cc43df40b4413ecacd8377,58cc43df40b4413ecacd8378,58cc43df40b4413ecacd8379,58cc43df40b4413ecacd837a,58cc43df40b4413ecacd837b,58cc43df40b4413ecacd837c,58cc43df40b4413ecacd837d,58cc43df40b4413ecacd837e,58cc43df40b4413ecacd837f,58cc43df40b4413ecacd8380,...,58cc43df40b4413ecacd91fc,58cc43df40b4413ecacd91fd,58cc43df40b4413ecacd91fe,58cc43df40b4413ecacd91ff,58cc43df40b4413ecacd9200,58cc43df40b4413ecacd9201,58cc43df40b4413ecacd9202,58cc43df40b4413ecacd9203,58cc43df40b4413ecacd9204,58cc43df40b4413ecacd9205


In [56]:
df_routes.shape[0]

3626

In [57]:
df_routes['name'].unique().shape[0]

3463

### Clean User data

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client.users_info
users_info = db.users_info
raw_data = users_info.find()
df_user_html = pd.DataFrame(list(raw_data))
df_user_html.head()

,_id,html,url,username
0,58d0bf3d40b44128d4d6df4e,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",/u/-jeremy-nelson//106959007,-Jeremy Nelson
1,58d0bf4140b44128d4d6df50,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",/u/1694kyle-bonnet//107129180,1694kyle Bonnet
2,58d0bf4440b44128d4d6df52,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",/u/20-kn//106349691,20 kN
3,58d0bf4840b44128d4d6df54,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",/u/419//106776142,419
4,58d0bf4b40b44128d4d6df56,"<!DOCTYPE html><html xmlns=""http://www.w3.org/...",/u/8jimi8//105855165,8jimi8


In [23]:
dict_list = []
_id = 0
for html in df_user_html['html']:
    dict_list.append(parse_user_page(html, _id))
    _id += 1
    if _id == 50:
        break

df_users = pd.DataFrame(dict_list)
df_users.head(50).T

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
age,NaN,NaN,NaN,33,NaN,44,NaN,25,26,NaN,...,NaN,26,31,NaN,NaN,22,NaN,26,NaN,42
aid_follows,NaN,NaN,C5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aid_leads,NaN,NaN,C3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,NaN,Seattle,Hawaii,Corvallis,NaN,Fort Thomas,San Diego,albuquerque,Seattle,Redmond,...,NaN,Providence,Portland,NaN,NaN,Seattle,NaN,San Jose,NaN,Bend
compliments,0,1,10,0,0,0,2,0,0,0,...,1,0,0,0,17,0,0,0,0,56
favorite_climbs,[ leading trad is rad],"[ high mountain woody, fridge left, easy in ...",[ none specified],[ outer space],[ none specified],[ none specified],"[ positive vibrations, whodunit, vertigo]",[ none specified],[ none specified],[ squamish & leavenworth],...,[ none specified],[ none specified],[ trad multi-pitch routes],[ none specified],[ none specified],"[ deception crags, goat wall]",[ none specified],[ none specified],[ none specified],"[ the grand wall, heinous cling, levitation ..."
female,NaN,0,0,0,NaN,0,1,1,0,0,...,NaN,0,1,NaN,NaN,0,NaN,0,0,1
ice_follows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI4,NaN,...,NaN,NaN,NaN,NaN,WI5,NaN,NaN,NaN,NaN,NaN
ice_leads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI3,NaN,...,NaN,NaN,NaN,NaN,WI3,NaN,NaN,NaN,NaN,NaN
id,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49


In [26]:
client = MongoClient('mongodb://localhost:27017/')
db = client.users
users = db.users
_id = 0
for html in df_user_html['html']:
    dict_list = parse_user_page(html, _id)
    users.insert_one(dict_list)
    _id += 1